In [67]:
%matplotlib QT5
import numpy as np
import matplotlib.pyplot as plt

import pyaccel
import pymodels
import idanalysis.optics as opt
from apsuite.optics_analysis import TuneCorr, OpticsCorr, CouplingCorr
from apsuite.orbcorr import OrbitCorr, CorrParams
from mathphys.functions import save_pickle, load_pickle

Load data

In [68]:
data_mach = load_pickle('SI_20_machines_seed_90560.pickle')

In [44]:
data_mach[4]['data'].keys()

dict_keys(['step_1', 'step_2', 'step_3', 'step_4', 'step_5', 'step_7', 'step_final'])

In [60]:
for mach in np.arange(0, 5):
    model = data_mach[mach]['model']
    orb = pyaccel.tracking.find_orbit4(model, indices='open')
    plt.plot(1e6*orb[2])
    print(np.std(1e6*orb[0]))

65.07535138809016
72.07021114041831
68.67786972792366
69.9779485112649
63.49930804803765


In [61]:
for mach in np.arange(0, 5):
    model = data_mach[mach]['model']
    twiss, *_ = pyaccel.optics.calc_twiss(model)
    etay = twiss.etay
    plt.plot(1e3*etay, label=mach)
plt.legend()

In [72]:
plt.figure()
for mach in np.arange(0, 20):
    model = data_mach[mach]['model']
    twiss, *_ = pyaccel.optics.calc_twiss(model)
    etay = twiss.etay
    plt.plot(1e3*etay)

In [ ]:
for mach in np.arange(0,20):
    print(data_mach[mach]['data'].keys())

In [62]:
mach = 0
model = data_mach[mach]['model']
ocorr_params = data_mach['orbcorr_params']
orb0 = data_mach[mach]['data']['step_1']['ref_orb']
orbcorr = OrbitCorr(model, 'SI', '4d', params=ocorr_params)
# kicks = orbcorr.get_kicks()
# kicks *= 0
# orbcorr.set_kicks(kicks)

jac_error = orbcorr.get_jacobian_matrix()
orbcorr.correct_orbit(
                jacobian_matrix=jac_error, goal_orbit=orb0)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/mamba_files/mamba/envs/sirius/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3913/4074428602.py", line 5, in <module>
    orbcorr = OrbitCorr(model, 'SI', '4d', params=ocorr_params)
  File "/home/gabriel/repos/apsuite/apsuite/orbcorr/orbit_correction.py", line 45, in __init__
  File "/home/gabriel/repos/apsuite/apsuite/orbcorr/calc_orbcorr_mat.py", line 26, in __init__
    self.fam_data = si.get_family_data(self.model)
  File "/home/gabriel/repos/pymodels/pymodels/SI_V25_04/families.py", line 352, in get_family_data
    section_map = get_section_name_mapping(lattice)
  File "/home/gabriel/repos/pymodels/pymodels/SI_V25_04/families.py", line 285, in get_section_name_mapping
    if b1[0] > start:
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent cal

In [ ]:
plt.plot()

In [ ]:
model_nominal = pymodels.si.create_accelerator()
model_nominal.cavity_on = False
model_nominal.radiation_on = 0
model_nominal.vchamber_on = False
ocorr_params = data_mach['orbcorr_params']
orbcorr = OrbitCorr(model_nominal, 'SI', '4d', params=ocorr_params)
jac = orbcorr.get_jacobian_matrix()

In [ ]:
plt.plot(jac_error.ravel()-jac.ravel())

In [ ]:
for mach in data_mach.keys():
    print(mach)

In [ ]:
params = data_mach['orbcorr_params']
print(params.minsingval)
print(params.maxnriters)
print(params.tolerance)

plot final COD for all machines

In [64]:
plt.figure()
for mach in np.arange(0,5):
    model = data_mach[mach]['model']
    orb0 = data_mach[mach]['data']['step_1']['ref_orb']
    orbcorr = OrbitCorr(model, 'SI', '4d', params=ocorr_params)
    jac_error = orbcorr.get_jacobian_matrix()
    orbcorr.correct_orbit(
                jacobian_matrix=jac, goal_orbit=orb0)
    orbf = orbcorr.get_orbit()
    cod = orbf-orb0
    print('{:.2f} %'.format(np.std(1e6*cod)))
    if np.max(1e6*cod) >= 100:
        print('error mach:', mach)
    plt.plot(1e6*cod)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/mamba_files/mamba/envs/sirius/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3913/90285897.py", line 5, in <module>
    orbcorr = OrbitCorr(model, 'SI', '4d', params=ocorr_params)
  File "/home/gabriel/repos/apsuite/apsuite/orbcorr/orbit_correction.py", line 45, in __init__
  File "/home/gabriel/repos/apsuite/apsuite/orbcorr/calc_orbcorr_mat.py", line 26, in __init__
    self.fam_data = si.get_family_data(self.model)
  File "/home/gabriel/repos/pymodels/pymodels/SI_V25_04/families.py", line 352, in get_family_data
    section_map = get_section_name_mapping(lattice)
  File "/home/gabriel/repos/pymodels/pymodels/SI_V25_04/families.py", line 285, in get_section_name_mapping
    if b1[0] > start:
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call 

In [70]:
plt.figure()
for mach in np.arange(0, 20):
    orb0 = data_mach[mach]['data']['step_final']['ref_orb']
    cod = orb0
    codx = cod[0:int(len(cod)/2)]
    cody = cod[int(len(cod)/2):]
    rmsx = np.std(1e6*codx)
    rmsy = np.std(1e6*cody)
    print('rms x: {:.3f} [um]'.format(rmsx))
    print('rms y: {:.3f} [um]'.format(rmsy))
    plt.plot(1e6*cod)

rms x: 46.416 [um]
rms y: 46.738 [um]
rms x: 52.012 [um]
rms y: 46.522 [um]
rms x: 47.218 [um]
rms y: 50.746 [um]
rms x: 51.395 [um]
rms y: 46.169 [um]
rms x: 49.111 [um]
rms y: 46.179 [um]
rms x: 49.159 [um]
rms y: 52.212 [um]
rms x: 45.958 [um]
rms y: 45.795 [um]
rms x: 48.792 [um]
rms y: 50.500 [um]
rms x: 48.096 [um]
rms y: 50.995 [um]
rms x: 50.840 [um]
rms y: 47.644 [um]
rms x: 45.712 [um]
rms y: 50.213 [um]
rms x: 49.604 [um]
rms y: 42.102 [um]
rms x: 49.187 [um]
rms y: 51.659 [um]
rms x: 49.493 [um]
rms y: 45.922 [um]
rms x: 50.844 [um]
rms y: 48.536 [um]
rms x: 46.591 [um]
rms y: 47.918 [um]
rms x: 51.617 [um]
rms y: 49.250 [um]
rms x: 53.506 [um]
rms y: 43.650 [um]
rms x: 49.531 [um]
rms y: 48.207 [um]
rms x: 50.631 [um]
rms y: 49.181 [um]


In [71]:
plt.figure()
for mach in np.arange(0, 20):
    orbf = data_mach[mach]['data']['step_final']['orbit']
    orb0 = data_mach[mach]['data']['step_final']['ref_orb']
    cod = orbf-orb0
    codx = cod[0:int(len(cod)/2)]
    cody = cod[int(len(cod)/2):]
    rmsx = np.std(1e6*codx)
    rmsy = np.std(1e6*cody)
    print('rms x: {:.3f} [um]'.format(rmsx))
    print('rms y: {:.3f} [um]'.format(rmsy))
    plt.plot(1e6*cod)

rms x: 17.656 [um]
rms y: 0.788 [um]
rms x: 18.616 [um]
rms y: 0.301 [um]
rms x: 18.427 [um]
rms y: 1.043 [um]
rms x: 16.501 [um]
rms y: 0.289 [um]
rms x: 15.788 [um]
rms y: 0.363 [um]
rms x: 19.785 [um]
rms y: 0.359 [um]
rms x: 19.033 [um]
rms y: 0.778 [um]
rms x: 21.973 [um]
rms y: 0.346 [um]
rms x: 19.583 [um]
rms y: 1.023 [um]
rms x: 17.908 [um]
rms y: 0.395 [um]
rms x: 16.952 [um]
rms y: 0.426 [um]
rms x: 17.651 [um]
rms y: 0.326 [um]
rms x: 16.151 [um]
rms y: 0.437 [um]
rms x: 18.853 [um]
rms y: 4.841 [um]
rms x: 17.222 [um]
rms y: 0.328 [um]
rms x: 19.109 [um]
rms y: 0.373 [um]
rms x: 19.639 [um]
rms y: 0.845 [um]
rms x: 17.260 [um]
rms y: 0.438 [um]
rms x: 16.204 [um]
rms y: 0.455 [um]
rms x: 19.881 [um]
rms y: 0.338 [um]


Plot final correctors kicks for all machines

In [ ]:
plt.figure()
for mach in np.arange(0,20):
    kicks7 = data_mach[mach]['data']['step_final']['corr_kicks'][:-1]
    # kicks6 = data_mach[mach]['data']['step_6']['corr_kicks'][:-1]
    # deltakick = kicks7 - kicks6
    plt.plot(1e6*kicks7)

Verify if final COD is in the null-space of orb respmat

In [ ]:
plt.figure()
minsingval = 0.2
respmats = load_pickle('respmats.pickle')
orbmat = respmats['orbmat']


umat, smat, vmat = np.linalg.svd(orbmat, full_matrices=False)
idx = smat > minsingval
ismat = np.zeros(smat.shape, dtype=float)
ismat[idx] = 1/smat[idx]
ismat = np.dot(vmat.T*ismat, umat.T)
for mach in np.arange(0,20):
    orbf = data_mach[mach]['data']['step_final']['orbit']
    orb0 = data_mach[mach]['data']['step_final']['ref_orb']
    kicks7 = data_mach[mach]['data']['step_final']['corr_kicks'][:-1]
    cod = orbf-orb0
    dkicks = -1*np.dot(ismat, cod)
    plt.plot(1e6*dkicks)

Plot final beta-beating for all machines

In [52]:
si = pymodels.si.create_accelerator()
mib = pyaccel.lattice.find_indices(si, 'fam_name', 'mib')
mip = pyaccel.lattice.find_indices(si, 'fam_name', 'mip')
mia = pyaccel.lattice.find_indices(si, 'fam_name', 'mia')
bpms = pyaccel.lattice.find_indices(si, 'fam_name', 'BPM')
idcs_nominal = mia + mib + mip + bpms
idcs_nominal.sort()
twiss0, *_ = pyaccel.optics.calc_twiss(si)
betax0 = twiss0.betax[idcs_nominal]
betay0 = twiss0.betay[idcs_nominal]
etax0 = twiss0.etax[idcs_nominal]
etay0 = twiss0.etay

bbtx = np.zeros(20)
bbty = np.zeros(20)
fig, axs = plt.subplots(2)
for mach in range(20):
    bbeatx = 1e2*data_mach[mach]['data']['step_final']['betabeatingx'][idcs_nominal]
    bbeaty = 1e2*data_mach[mach]['data']['step_final']['betabeatingy'][idcs_nominal]
    bbtx[mach] = np.std(bbeatx)
    bbty[mach] = np.std(bbeaty)
    print('beta beating x: {:.2f} %'.format(np.std(bbeatx)))
    print('beta beating y: {:.2f} %'.format(np.std(bbeaty)))
    print()
    axs[0].plot(bbeatx, label=mach)
    axs[1].plot(bbeaty, label=mach)
print('beta beating x average: {:.2f} %'.format(np.mean(bbtx)))
print('beta beating y average: {:.2f} %'.format(np.mean(bbty)))

IndexError: index 2097 is out of bounds for axis 0 with size 2097

In [ ]:
fig, axs = plt.subplots(2)
etx = np.zeros(20)
for mach in np.arange(20):
    model = data_mach[mach]['model']
    # get indices of bpms and straight sections center
    mib = pyaccel.lattice.find_indices(model, 'fam_name', 'mib')
    mip = pyaccel.lattice.find_indices(model, 'fam_name', 'mip')
    mia = pyaccel.lattice.find_indices(model, 'fam_name', 'mia')
    bpms = pyaccel.lattice.find_indices(model, 'fam_name', 'BPM')
    idcs = mia + mib + mip + bpms
    idcs.sort()

    twiss, *_ = pyaccel.optics.calc_twiss(model)
    etax = twiss.etax[idcs]
    etay = twiss.etay
    etax_diff = 1e3*(etax - etax0)

    etx[mach] = np.std(etax_diff)
    print('eta x diff: {:.2f} mm'.format(np.std(etax_diff)))
    print('eta y : {:.2f} mm'.format(np.std(1e3*etay)))
   
    print()
    axs[0].plot(1e3*etax_diff)
    axs[1].plot(1e3*etay)

In [53]:
for mach in np.arange(5):
    model = data_mach[mach]['model']
    ed_tang, *_ = pyaccel.optics.calc_edwards_teng(model)
    min_tunesep, ratio =\
            pyaccel.optics.estimate_coupling_parameters(ed_tang)
    print('Minimum tune separation {:.2f} %:'.format(100*min_tunesep))

Minimum tune separation 1.26 %:
Minimum tune separation 0.97 %:
Minimum tune separation 0.81 %:
Minimum tune separation 0.91 %:
Minimum tune separation 0.35 %:


In [ ]:
fig, axs = plt.subplots(2)

for mach in range(20):
    twiss = data_mach[mach]['data']['step_final']['twiss']
    axs[0].plot(1e3*twiss.etax, label=mach)
    axs[1].plot(1e3*twiss.etay, label=mach)
axs[0].legend()

Calc DA

In [187]:
nrconfigs = 5
data = load_pickle('dynapt_90560')
x0, y0 = data[0], data[1]

In [188]:
dim = (nrconfigs + 1, ) + np.shape(x0)
x, y = np.zeros(dim), np.zeros(dim)
x[0, :], y[0, :] = x0, y0
for i in range(0, nrconfigs, 1):
    config = str(i)
    data = load_pickle('dynapt_90560_config' + config)
    x[i+1, :], y[i+1, :] = data[0], data[1]

In [189]:
plt.figure()
labels = ['bare lattice', 'w = 24 mm', 'w = 26 mm', 'w = 28 mm', 'w = 30 mm', 'w = 32 mm']
for i in range(0, nrconfigs+1, 1):
    xi, yi = x[i], y[i]
    for mach in np.arange(0, 20):
        if mach == 0:
            plt.plot(1e3*xi[mach,:], 1e3*yi[mach,:], '--', 
                     color=plt.cm.jet(i/nrconfigs),
                     label=labels[i])
        else:
            plt.plot(1e3*xi[mach,:], 1e3*yi[mach,:], '--', 
                     color=plt.cm.jet(i/nrconfigs))
       
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.legend()
plt.show()
# plt.savefig(title+'.png')

In [183]:
nrtheta = 9
x_mean, y_mean = np.zeros((nrconfigs+1, nrtheta)), np.zeros((nrconfigs+1, nrtheta))
x_rms, y_rms = np.zeros((nrconfigs+1, nrtheta)), np.zeros((nrconfigs+1, nrtheta))

for i in range(nrconfigs + 1):
    x_mean[i, :] = np.mean(x[i], axis=0)
    y_mean[i, :] = np.mean(y[i], axis=0)
    x_rms[i, :] = np.std(x[i], axis=0)
    y_rms[i, :] = np.std(y[i], axis=0)

plt.figure()
plt.suptitle('avg DA +/- rms')
for i in range(0, nrconfigs + 1, 1):
    plt.plot(1e3*x_mean[i, :], 1e3*y_mean[i, :], '.-', 
         color=plt.cm.jet(i/nrconfigs), 
         label=labels[i])
    plt.plot(1e3*(x_mean[i, :]-x_rms[i, ]), 
             1e3*(y_mean[i, :]+y_rms[i, :]), '--',
             color=plt.cm.jet(i/nrconfigs))
    plt.plot(1e3*(x_mean[i, :]+x_rms[i, :]), 
             1e3*(y_mean[i, :]-y_rms[i, :]), '--', 
             color=plt.cm.jet(i/nrconfigs))
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.legend()
plt.grid()
plt.show()
# plt.savefig(title + ' mean.png')